# Módulo 14: Flujos de trabajo (workflows/pipelines) bioinformático de NGS

## Flujo de trabajo bioinformático

El flujo de trabajo o pipeline bioinformático NGS es un algoritmo predefinido que pretende combinar otros programas o scripts independientes, para guiar y procesar progresivamente datos de secuencias masivas, junto con sus metadatos asociados, a través de una serie de transformaciones utilizando múltiples componentes de software, bases de datos y entornos operativos (1,2) Por ejemplo, uno querría escribir un programa que ejecute Trimmomatic, FastQC, BWA/SPAdes, Kraken, Samtools, Picard, GATK y Bcftools utilizando un único comando.

![Intro](images/pipeline.png)

*Fig 1: Flujo de trabajo simplificado de análisis bioinformático de secuenciación de nueva generación (NGS). Los recuadros en amarillo representan las herramientas de ejecución utilizadas para procesar los datos de NGS. Los recuadros en verde representan los datos de salida. QC: Control de calidad. BWA: Alineador Burrows-Wheeler. SMALT: Herramienta de mapeo y alineación de secuencias. GATK: kit de herramientas de análisis del genoma. SSAHA: Búsqueda de Secuencias y Alineamiento por Algoritmo Hashing. BAM: Mapa de alineamiento binario. SAM: Mapa de alineamiento de secuencias. VCF: Formato de llamada de variantes. CSV: valores separados por comas. GFF: Formato general de características.*

## Gestores de flujos de trabajo bioinformáticos

Un gestor de flujos de trabajo bioinformáticos (también conocido como motor de flujos de trabajo, marco de flujos de trabajo o gestor de flujos de trabajo) es un sistema para construir y ejecutar flujos de trabajo, y debe ser capaz de optimizar el uso de recursos, gestionar la instalación y las versiones de software, y ejecutarse en diferentes plataformas informáticas, permitiendo la portabilidad y el intercambio de flujos de trabajo (2-4).

### 1. Gestores tradicionales de flujos de trabajo bioinformáticos

Históricamente, se han utilizado scripts y makefiles para desarrollar pipelines/flujos de trabajo computacionales (2,4). Los scripts permiten el uso de variables y lógica condicional para construir pipelines flexibles, sin embargo, tienden a ser muy frágiles, careciendo de soporte para (2,5):

1. dependencias, que es la capacidad de actualizar los archivos posteriores cuando se actualizan los archivos anteriores;
2. y reentrada, que es la capacidad de un programa para continuar donde lo dejó si se interrumpe, eliminando así la necesidad de reiniciar desde el principio.

GNU Make es una herramienta Unix que ejecuta automáticamente una serie de comandos para construir un archivo destino a partir de archivos fuente y especifica una lista de dependencias del archivo destino leyendo un makefile (2,6). Esta herramienta introdujo el concepto de caracteres comodín, lo que significa que un único nombre de archivo puede especificar muchos archivos utilizando caracteres comodín [i.e., ‘*', ‘?' and ‘[...]'] (2,7). Por ejemplo, *.fasta' especifica una lista de todos los archivos cuyos nombres terminan en '.fasta' dentro del directorio de trabajo (7). Aunque Make se puede utilizar para cualquier flujo de trabajo, originalmente fue pensado para coordinar la compilación de software (6) y por lo tanto tiene las siguientes limitaciones cuando se utiliza para pipelines científicos:

1. la distribución de tareas que se pueden ejecutar en paralelo en varios nodos de un clúster no es fácil de hacer dentro del marco de Make, ya que carece de un soporte integrado para la computación distribuida (1,2).

2. el uso de otros medios para asociar entradas a salidas que no sea el nombre exacto del archivo es un reto ya que la sintaxis de Make está restringida a un comodín por regla y no permite tablas de búsqueda (1,2).

3. una lógica más sofisticada es difícil de implementar y por lo tanto poco práctica para los análisis bioinformáticos modernos (1,2).

![Intro](images/esq.png)

*Fig. 2: Esquema de un gestor de flujos de trabajo tradicional. Suele estar acoplado al entorno informático local y no es flexible a los cambios de versiones de software o datos.*

### 2. Gestores modernos de flujos de trabajo bioinformáticos

Los gestores de flujos de trabajo modernos automatizan los análisis computacionales encadenando tareas individuales de procesamiento de datos en cadenas cohesionadas, y compensan las limitaciones de los gestores de flujos de trabajo tradicionales en cuanto a sintaxis, supervisión, procesamiento paralelo, visualización, control de versiones y elaboración de informes resumidos(2,3). Estos modernos gestores de flujos de trabajo son la interfaz entre el analista y el entorno computacional y suelen englobar:

- las especificaciones del lenguaje de flujo de trabajo para interconectar los ejecutables de análisis; y
- el motor de ejecución para despachar las tareas y gestionar las dependencias de la infraestructura informática (Fig 3).

![Intro](images/esq2.png)

*Fig. 3: Visión general de un gestor de flujo de trabajo bioinformático moderno. Los elementos del gestor de flujo de trabajo se ilustran en el recuadro marrón punteado. Las herramientas de análisis y el almacenamiento de datos pueden estar en contenedores y también almacenados en la computación en nube. Los gestores de flujos de trabajo pueden ejecutarse en cualquiera de los tres entornos informáticos que se muestran en el recuadro morado punteado, es decir, en un ordenador local (LC), en computación de alto rendimiento (HPU) o en computación en nube. Los pasos de ejecución se describen en el recuadro de puntos rosa. Los datos de entrada pueden ser archivos fastq y secuencias de referencia. Los pasos 1 - 4 son el análisis bioinformático del flujo de trabajo (que incluye: recorte de adaptadores, control de calidad, mapeo/ensamblaje y caracterización in silico). Los datos de salida incluyen: informe de control de calidad y archivos del genoma (VCF, CSV, TAB, GFF, BAM, FASTA/FA). Lo que hace que un gestor de flujo de trabajo moderno sea más ventajoso/robusto que los gestores de flujo de trabajo tradicionales son los siguientes aspectos: portabilidad (descrita en el recuadro verde inferior), escalabilidad (descrita en el recuadro azul inferior), reentrada (descrita en el recuadro amarillo inferior) y procedencia de los datos (descrita en el recuadro naranja claro inferior)  (2,3,8–19). Los puntos de control de reentrada se muestran en las estrellas amarillas (estos puntos de control pueden estar en el paso de entrada de datos o en cualquier paso del proceso de ejecución).*

Existen varios gestores de flujos de trabajo en el mercado. A grandes rasgos, estos gestores de flujos de trabajo pueden clasificarse en cuatro clases (2,3):

**Gestores gráficos de flujos de trabajo**: ofrecen una interfaz para que los usuarios puedan arrastrar y soltar herramientas en los flujos de trabajo y encadenarlas, lo que permite la creación de complejas canalizaciones computacionales sin experiencia en programación. Algunos ejemplos son Galaxy y KNIME.

**Lenguaje específico del dominio (DSL)**: se trata de un lenguaje de programación desarrollado para satisfacer una necesidad específica dentro de un dominio concreto, lo que permite el despliegue rápido de flujos de trabajo computacionales reproducibles, robustos y portátiles. Algunos ejemplos son Nextflow, Snakemake, GenPIPES, bPIPES, Pachyderm.

**Basados en bibliotecas**: implementan sus sistemas de gestión de canalizaciones como una biblioteca de programación para un lenguaje de programación popular existente, como python, y aprovechan las herramientas existentes, el soporte de editores de texto y otras bibliotecas de programación del lenguaje. Algunos ejemplos son SciPipe y Luigi.

**Especificaciones de flujo de trabajo**: permiten separar el flujo de trabajo del entorno de ejecución, y pueden ejecutarse a través de gestores de flujo de trabajo o entornos de ejecución que cumplan los requisitos de hardware para el flujo de trabajo. Es adecuado para científicos/investigadores que desean desvincular su flujo de trabajo de un gestor de flujo de trabajo específico para permitir un mayor grado de portabilidad y uso compartido. Algunos ejemplos son Common Workflow Language (CWL), Workflow description Language (WDL).

En la **Tabla 1** se ofrece una visión general de estos gestores de flujos de trabajo.

### Cómo elegir un gestor de flujos de trabajo

La elección de un gestor de flujos de trabajo debe basarse en los siguientes aspectos:

- Las exigencias del desarrollo del pipeline y el nivel de conocimientos del usuario final (es decir, desarrolladores altamente cualificados frente a aquellos con una experiencia mínima en programación)(2,3). Los gestores gráficos de flujos de trabajo fáciles de usar, como Galaxy, y los repositorios de pipelines listos para usar permiten la ejecución de análisis complejos sin experiencia en programación (3). Los repositorios de pipelines listos para usar permiten compartir y ejecutar pipelines, lo que facilita la transparencia y la reproducibilidad de los flujos de trabajo. Por ejemplo, el proyecto nf-core proporciona un marco curado para los pipelines Nextflow con mejores prácticas específicas y un conjunto de estándares para la implementación de pipelines que garanticen su mantenimiento, documentación, portabilidad, escalabilidad y reproducibilidad (3,20).
    
- Reproducibilidad del gestor de flujos de trabajo: un gestor de flujos de trabajo sólido debería permitir la procedencia de los datos, la legibilidad, la portabilidad, aumentar la transparencia, permitir la sostenibilidad a largo plazo de los flujos de trabajo de análisis y ayudar a conseguir análisis computacionales fiables, accesibles, interoperables y reutilizables(2,3).
    
- Facilidad de integración de nuevas herramientas (2).
    
- Tipo de análisis de datos que se realiza: la investigación a gran escala y altamente repetitiva que requiere un alto grado de procedencia de los datos y control de versiones puede beneficiarse de los gestores de especificación de flujos de trabajo(2,3).
    
- Accesibilidad a los recursos HPC internos: los investigadores con acceso limitado a los recursos HPC pueden considerar los bancos de trabajo basados en la nube y la interfaz de programación de aplicaciones (API). Los gestores de flujos de trabajo basados en servidores ofrecen la ventaja añadida de una escalabilidad ilimitada y oportunidades de investigación colaborativa, aunque incurren en costes directos (2,3).


### Oportunidades

Dado que los gestores de flujos de trabajo se están compartiendo ampliamente con un mayor énfasis en garantizar la transparencia y la reproducibilidad, existe una creciente necesidad de estandarizar el lenguaje de descripción de flujos de trabajo, así como de garantizar una evaluación sistemática y cuantitativa de su solidez, memoria y requisitos de almacenamiento (2,3). Para que los códigos de los flujos de trabajo de análisis bioinformático tengan un impacto duradero y sean sostenibles, será necesaria una financiación específica a largo plazo que respalde las mejores prácticas de desarrollo, mantenimiento, normalización y evaluación de software (2,3).

|   Class                           |   Tool (Pipeline Initiative)         |   Ease of development  |   Ease of use  |   Portability                                                               |   Scalability  |   Expressiveness                   |   Resources            |
|-----------------------------------|--------------------------------------|------------------------|----------------|-----------------------------------------------------------------------------|----------------|------------------------------------|------------------------|
|   Graphical                       |   Galaxy   (Galaxy community)        |   ⬤⬤⬤                  |   ⬤⬤⬤          |   ⬤⬤⬤                                                                       |   ⬤⬤⬤          |   ⬤◯◯   Docker, Conda, Singularity |   Tutorials, Forums    |
|                         |   KNIME   (KNIME Hub)                |   N/A                  |   ⬤⬤⬤          |   ◯◯◯  GUI                                                                  |   ⬤⬤◐          |   ⬤◯◯                              |   Tutorials, Forums    |
|   Domain-specific Language (DSL)  |   Nextflow    (nf-core)              |   ⬤⬤⬤◐                 |   ⬤⬤◯          |   ⬤⬤⬤                                                                       |   ⬤⬤⬤          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|     |                 |                        |                |   Docker, Shifter, Singularity, Podman, Conda                               |                |   Groovy                           |                        |
|     |   Snakemake   (Snakemake-Workflows)  |   ⬤⬤⬤◐                 |   ⬤⬤◯          |   ⬤⬤◐                                                                       |   ⬤⬤⬤          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|     |     |                        |                |   Singularity, Conda (Docker containers can be executed using Singularity)  |                |   Python                           |                        |
|     |   GenPipes (GenPipes)                |   N/A                  |   ⬤⬤◯          |   ⬤⬤◯                                                                       |   ⬤⬤◯          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|     |                   |                        |                |   Docker, Singularity                                                       |                |   Python                           |                        |
|     |   bPipes                             |   ⬤⬤⬤⬤                 |   ⬤⬤◯          |   ⬤⬤◯                                                                       |   ⬤⬤◐          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|   |                                |                        |                |   Docker, Singularity                                                       |                |   Groovy                           |                        |
|     |   Pachyderm                          |   N/A                  |   ⬤⬤◯          |   ⬤◯◯                                                                       |   ⬤⬤◯          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|     |                             |                        |                |   JSON and YAML                                                             |                |   Docker                           |                        |
|   Library                         |   SciPipe                            |   N/A                  |   ⬤⬤◯          |   ◯◯◯                                                                       |   ⬤⬤◐          |   ⬤⬤⬤                              |   Tutorials, Forums    |
|                            |                               |                        |                |   Go                                                                        |                |                                    |                        |
|                           |   Luigi                              |   ⬤⬤⬤                  |   ⬤⬤◯          |   ⬤◯◯                                                                       |   ⬤⬤◐          |   ⬤⬤⬤                              |   Tutorial, Forum      |
|                            |                                 |                        |                |   Python                                                                    |                |   Docker                           |                        |
|   Workflow specification          |   WDL    (BioWDL, WARP)              |   N/A                  |   ⬤◯◯          |   ⬤⬤⬤                                                                       |   ⬤⬤◐          |   ⬤⬤◯                              |   Tutorials,   Forums  |
|             |                 |                        |                |   Cromwell                                                                  |                |   Docker, Singularity, udocker     |                        |
|             |   CWL                                |   N/A                  |   ⬤◯◯          |   ⬤⬤◐                                                                       |   ◯◯◯          |   ⬤⬤◯                              |   Tutorials,   Forums  |
|             |                                   |                        |                |   Cwltool                                                                   |                |   Docker, udocker                  |                        |
|             |   CWL/WDL/   Python                  |   ⬤⬤◐                  |   ⬤◯◯          |   ⬤◐◯                                                                       |   ⬤⬤⬤          |   ⬤⬤⬤                              |   Tutorials,   Forums  |
|             |                     |                        |                |   Toil                                                                      |                |   Docker                           |                        |

*Tabla 1. CWL: Common Workflow Language Specification. JSON: JavaScript Object Notation. WDL: Workflow description Language pronounced ‘widdle’. GUI: Graphical User Interface. N/A: Not Available*

## Referencias

1. Standards and Guidelines for Validating Next-Generation Sequencing Bioinformatics Pipelines: A Joint Recommendation of the Association for Molecular Pathology and the College of American Pathologists. J Mol Diagn. 2018 Jan 1;20(1):4–27.

2. Leipzig J. A review of bioinformatic pipeline frameworks. Brief Bioinform. 2017 May;18(3):530.

3. Wratten L, Wilm A, Göke J. Reproducible, scalable, and shareable analysis pipelines with bioinformatics workflow managers. Nat Methods. 2021 Sep 23;18(10):1161–8.

4. What is the difference between a Bioinformatics pipeline and workflow?. Bioinformatics Stack Exchange: https://bioinformatics.stackexchange.com/questions/7347/what-is-the-difference-between-a-bioinformatics-pipeline-and-workflow

5. Sussman GJ. Building Robust Systems. 2007. Available from: https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.1324&rep=rep1&type=pdf

6. Make - GNU Project - Free Software Foundation. Available from: https://www.gnu.org/software/make/

7. Writing Rules. Available from: https://web.mit.edu/gnu/doc/html/make_4.html

8. Ahmed AE, Allen JM, Bhat T, Burra P, Fliege CE, Hart SN, et al. Design considerations for workflow management systems use in production genomics research and the clinic. Sci Rep. 2021 Nov 4;11(1):1–18.

9. Abate P, Di Cosmo R, Treinen R, Zacchiroli S. A modular package manager architecture [Internet]. Vol. 55, Information and Software Technology. 2013. p. 459–74. Available from: http://dx.doi.org/10.1016/j.infsof.2012.09.002

10. Decan A, Mens T, Grosjean P. An empirical comparison of dependency network evolution in seven software packaging ecosystems [Internet]. Vol. 24, Empirical Software Engineering. 2019. p. 381–416. Available from: http://dx.doi.org/10.1007/s10664-017-9589-y

11. Gruening B, Sallou O, Moreno P, da Veiga Leprevost F, Ménager H, Søndergaard D, et al. Recommendations for the packaging and containerizing of bioinformatics software [Internet]. Vol. 7, F1000Research. 2019. p. 742. Available from: http://dx.doi.org/10.12688/f1000research.15140.2

12. Silver A. Software simplified [Internet]. Vol. 546, Nature. 2017. p. 173–4. Available from: http://dx.doi.org/10.1038/546173a

13. Love MI, Soneson C, Hickey PF, Johnson LK, Pierce NT, Shepherd L, et al. Tximeta: Reference sequence checksums for provenance identification in RNA-seq. PLoS Comput Biol. 2020 Feb;16(2):e1007664.

14. Kim YM, Poline JB, Dumas G. Experimenting with reproducibility: a case study of robustness in bioinformatics. Gigascience [Internet]. 2018 Jul 1;7(7). Available from: http://dx.doi.org/10.1093/gigascience/giy077

15. Black A, MacCannell DR, Sibley TR, Bedford T. Ten recommendations for supporting open pathogen genomic analysis in public health. Nat Med. 2020 Jun;26(6):832–41.

16. Langmead B, Nellore A. Cloud computing for genomic data analysis and collaboration. Nat Rev Genet. 2018 May;19(5):325.

17. Birger C, Hanna M, Salinas E, Neff J, Saksena G, Livitz D, et al. FireCloud, a scalable cloud-based platform for collaborative genome analysis: Strategies for reducing and controlling costs [Internet]. Available from: http://dx.doi.org/10.1101/209494

18. Han L, Canon LC, Casanova H, Robert Y, Vivien F. Checkpointing Workflows for Fail-Stop Errors [Internet]. 2017 IEEE International Conference on Cluster Computing (CLUSTER). 2017. Available from: http://dx.doi.org/10.1109/cluster.2017.14

19. Jackson M, Kavoussanakis K, Wallace EWJ. Using prototyping to choose a bioinformatics workflow management system. PLoS Comput Biol. 2021 Feb;17(2):e1008622.

20. Ewels PA, Peltzer A, Fillinger S, Patel H, Alneberg J, Wilm A, et al. The nf-core framework for community-curated bioinformatics pipelines. Nat Biotechnol. 2020 Mar;38(3):276–8.